# Assignment 4 Notebook
## Kaplan-Meier Analysis

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

# import data
data = pd.read_excel("../Data/RADCURE_Clinical_v04_20241219.xlsx")

# clean data
clean_data = data.copy()
missing_data = clean_data.isnull().sum() / len(clean_data) # calculate ratio of missing values per column

drop_cols = missing_data[missing_data > 0.6].index # create an index of columns that exceed threshold
clean_data.drop(drop_cols, 
                axis = 1, 
                inplace = True) # remove qualifying columns

clean_data.dropna(inplace = True)

relevant_data = clean_data[["Age", 
                           "Sex", 
                           "ECOG PS", 
                           "Smoking Status", 
                           "Ds Site", 
                           "Path", 
                           "Status", 
                           "Length FU"]]

for col in relevant_data.select_dtypes(include = [np.object_]).columns:
    relevant_data[col].replace(relevant_data[col].unique(),
                            range(0, len(relevant_data[col].unique())), 
                            inplace = True)
    relevant_data[col].astype(int)

# fit KME
kmf = KaplanMeierFitter()
kmf.fit(relevant_data['Length FU'], 
        event_observed = relevant_data['Status'])

# Plot the Kaplan-Meier curve
kmf.plot_survival_function()
plt.title('Kaplan-Meier Curve')
plt.xlabel('Time (Years)')
plt.ylabel('Survival Probability')
plt.show()

## Cox Proportional Hazards Model

## Random Survival Forest